## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [37]:
import pandas as pd

rate_100 = 100

ratings = pd.read_csv('ml-latest-small/ratings.csv')
#print(ratings)
#ratings['dateTime'] = pd.to_datetime(ratings['timestamp'])
#ratings['dateTime']
ratings['dateTime'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings['dateTime']
#print(ratings['dateTime'])

ratings_new = ratings.groupby('userId').agg({'dateTime': ['min', 'max'], 'rating': 'count'})
#print(ratings_new)
ratings_new['lifespan'] = (ratings_new['dateTime']['max'] - ratings_new['dateTime']['min']).astype('timedelta64[h]')

avg_lifetime = ratings_new[ratings_new['rating']['count']>rate_100]['lifespan'].mean()
#print(f"среднее время жизни пользователей: {avg_lifetime} часов")
print(f"Cреднее время жизни пользователей: {avg_lifetime:.0f} часов")

Cреднее время жизни пользователей: 11133 часов


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


    rzd - железнодорожные перевозки
    auto - автомобильные перевозки
    air - воздушные перевозки
    client_base - адреса клиентов


In [55]:
import pandas as pd

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
#print(rzd)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
#print(auto)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
#print(air)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
#print(client_base)
# fillna(0) - убирает NA и заменяет его на нужный символ\значение (как замена в Excel)
revenue = rzd.merge(auto,how='outer').merge(air,how='outer').fillna(0).head()
print(f"Таблица с выручкой без указания адреса:\n {revenue}\n")

revenue_add = (revenue.merge(client_base, on= 'client_id', how = 'left'))
print(f"Таблица с выручкой и указанием адреса:\n {revenue_add}")


Таблица с выручкой без указания адреса:
    client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0

Таблица с выручкой и указанием адреса:
    client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           0.0          0.0  Комсомольская 4
1        112       2810.0           0.0          0.0   Энтузиастов 8а
2        113      10283.0       57483.0          0.0  Левобережная 1а
3        114       5774.0          83.0          0.0          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

1. У каждого пользователя известен набор координат.
А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга.
Как это отразится на вопросе расчетах пункта 1?

3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [56]:
print('1. Мы можем взять координаты магазина, для подтверждения, что пользователь находился там или рядом; Так же потребуются данные процессинга карт(банк)\магазина продавшего товар потребителю\ОФД\производителя кассовой техники\разработчика ТУС для подтверждения факта покупки; Как дополнительное решение, или альтернатива процессингу и прочему, то данные по поводу мак-адреса клиента, когда он подходит к кассе на определённое расстояние, его устройство может быть обнаружено датчиком bluetooth\wifi, который считывет все мак-ажреса устройств находящихся в радиусе кассы (1-2 метра))\n')
print('2. Это способно исказить или сильно ухудшить качество данных для анализа, нам скорее-всего придётся укрупнить географические объеты для корректной работы с пользователем; С другой стороны, путём рассчёта насколько удалены и где находятся данные точки, мы можем найти "области интереса" у данного пользователя и более адресно с ним работать.\n')
print('3. Тип объетка, в котором находился пользователь, что может помочь лучше выявлять привычки и увлечения пользвателя; Регион; Город; Страна; Район; Частота посещения данного места; Сколько времени там пробыл пользователь; Организации, расположенные по данному адресу; Другие пользователи, рядом или на том же месте.')

1. Мы можем взять координаты магазина, для подтверждения, что пользователь находился там или рядом; Так же потребуются данные процессинга карт(банк)\магазина продавшего товар потребителю\ОФД\производителя кассовой техники\разработчика ТУС для подтверждения факта покупки; Как дополнительное решение, или альтернатива процессингу и прочему, то данные по поводу мак-адреса клиента, когда он подходит к кассе на определённое расстояние, его устройство может быть обнаружено датчиком bluetooth\wifi, который считывет все мак-ажреса устройств находящихся в радиусе кассы (1-2 метра))

2. Это способно исказить или сильно ухудшить качество данных для анализа, нам скорее-всего придётся укрупнить географические объеты для корректной работы с пользователем; С другой стороны, путём рассчёта насколько удалены и где находятся данные точки, мы можем найти "области интереса" у данного пользователя и более адресно с ним работать.

3. Тип объетка, в котором находился пользователь, что может помочь лучше выявл